In [1]:
!pip install ultralytics
!pip install -U ipywidgets
from ultralytics import YOLO
import torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 58.8 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.9
    Uninstalling widgetsnbextension-3.6.9:
      Successfully uninstalled widgetsnbextension-3.6.9
  Attempting uninstall: jupyterlab-widgets
    Found existing installation: jupyterlab_widgets 3.0.11
    Uninstalling jupyterlab_widgets-3.0.11:
      Successfully uninstalled jupyterlab_widgets-3.0.11
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. Thi

In [2]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

model.train(
    data='/kaggle/input/cv-project-files2/dataset.yaml',
    epochs=75,                            # Number of training epochs
    imgsz=640,                            # Image size
    batch=32,                             # batch size
    lr0=0.001,                             # initial learning rate
    augment=True,                         # no data augmentation
    dropout=0.2       ,                    # dropout to reduce overfitting
    device=[0,1]
)


100%|██████████| 6.25M/6.25M [00:00<00:00, 133MB/s]


Ultralytics 8.3.37 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
                                                 CUDA:1 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/kaggle/input/cv-project-files2/dataset.yaml, epochs=75, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=[0, 1], workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.2, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save

100%|██████████| 755k/755k [00:00<00:00, 23.0MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 98.7MB/s]


...

                   all       1073       1573       0.85      0.805      0.854      0.467

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      75/75      2.16G     0.9291     0.3453      1.036         18        640: 100%|██████████| 796/796 [03:34<00:00,  3.72it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:06<00:00,  4.88it/s]


                   all       1073       1573      0.851      0.805      0.855      0.468

75 epochs completed in 4.805 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.37 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
                                                 CUDA:1 (Tesla T4, 15095MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:10<00:00,  3.32it/s]


                   all       1073       1573      0.844      0.761      0.838      0.457
Speed: 0.1ms preprocess, 5.4ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/detect/train


In [3]:
#from ultralytics import YOLO
# resume training
#model = YOLO('/kaggle/working/runs/detect/train/weights/last.pt')
#model.train(resume=True)

In [4]:
image_folder = '/kaggle/input/large-license-plate-dataset/images/test/'
model = YOLO('/kaggle/working/runs/detect/train/weights/best.pt')

results = model(image_folder,stream=False)
output_folder = '/kaggle/working/test'
ok = 0
skip = 0
for image in results:
    device = image.boxes.xywh.device
    suitable_crops = torch.empty((0,6), device=device)
    for i in range(len(image)):
        x, y, w, h = image.boxes.xywh[i].tolist() 
        if w > 50 and h > 20:
            conf = image.boxes.conf[i]
            keep_crop = torch.tensor([[x-w/2,y-h/2,x+w/2,y+h/2,conf,0]],device=device)
            suitable_crops = torch.cat((suitable_crops,keep_crop))
            ok+=1
        else:
            skip+=1
    image.update(suitable_crops)
    image.save_crop(output_folder)

print("ok",ok,"skip",skip)


image 1/386 /kaggle/input/large-license-plate-dataset/images/test/003a5aaf6d17c917.jpg: 448x640 1 license_plate, 35.5ms
image 2/386 /kaggle/input/large-license-plate-dataset/images/test/00723dac8201a83e.jpg: 480x640 2 license_plates, 37.2ms
image 3/386 /kaggle/input/large-license-plate-dataset/images/test/008637722500f239.jpg: 480x640 2 license_plates, 6.1ms
image 4/386 /kaggle/input/large-license-plate-dataset/images/test/0170ea8e1a33375a.jpg: 448x640 2 license_plates, 7.0ms
image 5/386 /kaggle/input/large-license-plate-dataset/images/test/017527da8bfeb97d.jpg: 448x640 1 license_plate, 6.3ms
image 6/386 /kaggle/input/large-license-plate-dataset/images/test/02a6ef3d9bd68e91.jpg: 480x640 2 license_plates, 7.0ms
image 7/386 /kaggle/input/large-license-plate-dataset/images/test/03b7b71e1ffcb7a8.jpg: 448x640 1 license_plate, 6.8ms
image 8/386 /kaggle/input/large-license-plate-dataset/images/test/044417ca6134604f.jpg: 448x640 1 license_plate, 6.1ms
image 9/386 /kaggle/input/large-license-p

In [5]:
# to save the run : zip the folder from this command and then download the zip file
#import shutil
#shutil.make_archive("train_20", 'zip', "/kaggle/working/runs/detect/train")